In [1]:
import os
os.getcwd()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab-Notebooks/LLM-Project

/content/drive/MyDrive/Colab-Notebooks/LLM-Project


In [3]:
# Run this in a fresh Colab cell **before** you import datasets / transformers
!pip install -U -q datasets huggingface_hub fsspec

# then RESTART the runtime (Runtime ▸ Restart runtime) or:
# import importlib, sys; importlib.reload(sys.modules['datasets'])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_6

In [4]:
from datasets import load_dataset

ds = load_dataset('SetFit/20_newsgroups')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/734 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/8.91M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11314 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7532 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 7532
    })
})

In [6]:
ds['train'][0]

{'text': 'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.',
 'label': 7,
 'label_text': 'rec.autos'}

In [7]:
ds['train'].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_text': Value(dtype='string', id=None)}

In [8]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [9]:
# def clean_formatting(text):
#     text = "".join([char for char in text if char not in "\n"])
#     text = "".join([char for char in text if char not in string.punctuation])
#     return text


def clean_formatting(text):
    translator = str.maketrans("\n", ' ' * len("\n"))
    cleaned_text = text.translate(translator)
    cleaned_text = "".join([char for char in cleaned_text if not char.isdigit()])
    translator2 = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    cleaned_text = cleaned_text.translate(translator2)
    final_text = " ".join(cleaned_text.split()).lower()
    return final_text


In [10]:
import pandas as pd
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])

In [11]:
ds_train.head()

,text,label,label_text
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space


In [12]:
ds_train['label'].value_counts()

,count
label,
10,600
15,599
8,598
9,597
11,595
13,594
7,594
14,593
5,593


In [13]:
ds_train['label_text'].value_counts()

,count
label_text,
rec.sport.hockey,600
soc.religion.christian,599
rec.motorcycles,598
rec.sport.baseball,597
sci.crypt,595
sci.med,594
rec.autos,594
sci.space,593
comp.windows.x,593


In [14]:
text = ds_train['text'][0]
print(text)

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [15]:
text_no_punct = clean_formatting(text)
print(text_no_punct)

i was wondering if anyone out there could enlighten me on this car i saw the other day it was a door sports car looked to be from the late s early s it was called a bricklin the doors were really small in addition the front bumper was separate from the rest of the body this is all i know if anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please e mail


In [16]:
ds_train['text_clean'] = ds_train['text'].apply(lambda x: clean_formatting(x))

In [17]:
ds_train.head()

,text,label,label_text,text_clean
0,I was wondering if anyone out there could enli...,7,rec.autos,i was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware,a fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware,well folks my mac plus finally gave up the gho...
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics,do you have weitek s address phone number i d ...
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space,from article cowcb np world std com by tombake...


In [18]:
# for lemmatization and POS tagging
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [19]:
def tokenize(text):
    tokens = text.lower().split()
    tokens = [x for x in tokens if len(x) > 1]
    return tokens

In [20]:
import spacy

# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Process the text using spaCy
# Lemmatization
ds_train["text_lemm"] = ds_train['text_clean'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))

# Extract lemmatized tokens
ds_train['tokens'] = ds_train['text_lemm'].apply(lambda x: tokenize(x))

In [21]:
ds_train.head()

,text,label,label_text,text_clean,text_lemm,tokens
0,I was wondering if anyone out there could enli...,7,rec.autos,i was wondering if anyone out there could enli...,I be wonder if anyone out there could enlighte...,"[be, wonder, if, anyone, out, there, could, en..."
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware,a fair number of brave souls who upgraded thei...,a fair number of brave soul who upgrade their ...,"[fair, number, of, brave, soul, who, upgrade, ..."
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware,well folks my mac plus finally gave up the gho...,well folk my mac plus finally give up the ghos...,"[well, folk, my, mac, plus, finally, give, up,..."
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics,do you have weitek s address phone number i d ...,do you have weitek s address phone number I d ...,"[do, you, have, weitek, address, phone, number..."
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space,from article cowcb np world std com by tombake...,from article cowcb np world std com by tombake...,"[from, article, cowcb, np, world, std, com, by..."


In [22]:
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root

True

In [23]:
from nltk.corpus import stopwords

In [24]:
stop_words = stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stop_words]

    return text

In [25]:
ds_train['tokens_nostop'] = ds_train['tokens'].apply(lambda x: remove_stopwords(x))

In [26]:
ds_train.head()

,text,label,label_text,text_clean,text_lemm,tokens,tokens_nostop
0,I was wondering if anyone out there could enli...,7,rec.autos,i was wondering if anyone out there could enli...,I be wonder if anyone out there could enlighte...,"[be, wonder, if, anyone, out, there, could, en...","[wonder, anyone, could, enlighten, car, see, d..."
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware,a fair number of brave souls who upgraded thei...,a fair number of brave soul who upgrade their ...,"[fair, number, of, brave, soul, who, upgrade, ...","[fair, number, brave, soul, upgrade, si, clock..."
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware,well folks my mac plus finally gave up the gho...,well folk my mac plus finally give up the ghos...,"[well, folk, my, mac, plus, finally, give, up,...","[well, folk, mac, plus, finally, give, ghost, ..."
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics,do you have weitek s address phone number i d ...,do you have weitek s address phone number I d ...,"[do, you, have, weitek, address, phone, number...","[weitek, address, phone, number, like, get, in..."
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space,from article cowcb np world std com by tombake...,from article cowcb np world std com by tombake...,"[from, article, cowcb, np, world, std, com, by...","[article, cowcb, np, world, std, com, tombaker..."


In [31]:
def clean_data(text):
  text_clean = clean_formatting(text)
  # Lemmatization
  text_lemm = " ".join([y.lemma_ for y in nlp(text_clean)])

  # Extract lemmatized tokens
  tokens = tokenize(text_lemm)
  tokens_nostop = remove_stopwords(tokens)

ds_test['tokens_nostop'] = ds_test['text'].apply(lambda x: clean_data(x))

In [32]:
from datasets import Dataset, DatasetDict
# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)
# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
# view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'text_clean', 'text_lemm', 'tokens', 'tokens_nostop'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'tokens_nostop'],
        num_rows: 7532
    })
})

In [33]:
# Save to an on‑disk format that preserves all columns & metadata
target_dir = "/content/drive/MyDrive/Colab-Notebooks/LLM-Project/datasets/20_newsgroups_ds"
new_ds.save_to_disk(target_dir)
print("Saved to", target_dir)

Saving the dataset (0/1 shards):   0%|          | 0/11314 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7532 [00:00<?, ? examples/s]

Saved to /content/drive/MyDrive/Colab-Notebooks/LLM-Project/datasets/20_newsgroups_ds
